In [5]:
import arcpy
import random
import math
import numpy as np
import time
from datetime import datetime
import os
import csv
import psutil

### Environments

In [1]:
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"D:\ECHO\CDL\ZonalEntropyIndex\Default.gdb"
arcpy.env.parallelProcessingFactor = "100%"

### Arguments

In [11]:
fishnet_fc = r"D:\ECHO\US_Fishnet.shp"
raster_path = r"D:\ECHO\CDL\CDL_RasterMgmt\Default.gdb\CDL_USA_2011"
target_values = [141, 142, 143, 152, 176, 190, 195, 87, 121]  # Greenspace, incl. 121 - dev. open space
# target_values = [83, 87, 111, 190, 195]  # Blue space
residential_values = [122] # Developed, low intensity
target_polys = 1_000_000  # Target polygons per tile
output_gdb = r"D:\ECHO\CDL\ZonalEntropyIndex\Default.gdb"
output_csv_path = r"D:\ECHO\EI_2011.csv"

### Functions

In [3]:
# Function to add fields to input feature class
def add_fields_to_fc(fc):
    fields = [f.name for f in arcpy.ListFields(fc)]

    # Add OrigOID field to store original OIDs if it doesn't exist
    if "OrigOID" not in fields:
        arcpy.management.AddField(fc, "OrigOID", "LONG")
        with arcpy.da.UpdateCursor(fc, ["FID", "OrigOID"]) as cursor:
            for row in cursor:
                row[1] = row[0]  # Copy FID to OrigOID
                cursor.updateRow(row)
        print("OrigOID field added and populated.")

# Function to split the fishnet into tiles
def split_fishnet_by_count(fishnet_fc, target_polys, output_gdb):
    total_polys = int(arcpy.management.GetCount(fishnet_fc)[0])
    tiles_needed = math.ceil(total_polys / target_polys)
    num_cols = num_rows = int(math.ceil(math.sqrt(tiles_needed)))

    extent = arcpy.Describe(fishnet_fc).extent
    xmin, ymin, xmax, ymax = extent.XMin, extent.YMin, extent.XMax, extent.YMax
    tile_width = (xmax - xmin) / num_cols
    tile_height = (ymax - ymin) / num_rows

    tile_list = []
    fishnet_layer = "fishnet_layer"
    arcpy.management.MakeFeatureLayer(fishnet_fc, fishnet_layer)

    for i in range(num_rows):
        for j in range(num_cols):
            x_min_tile = xmin + j * tile_width
            y_min_tile = ymin + i * tile_height
            x_max_tile = x_min_tile + tile_width
            y_max_tile = y_min_tile + tile_height

            tile_extent = arcpy.Polygon(arcpy.Array([
                arcpy.Point(x_min_tile, y_min_tile),
                arcpy.Point(x_max_tile, y_min_tile),
                arcpy.Point(x_max_tile, y_max_tile),
                arcpy.Point(x_min_tile, y_max_tile),
                arcpy.Point(x_min_tile, y_min_tile)
            ]))

            arcpy.management.SelectLayerByLocation(fishnet_layer, "INTERSECT", tile_extent)
            output_fc = f"{output_gdb}\\Tile_{i}_{j}"
            arcpy.management.CopyFeatures(fishnet_layer, output_fc)
            tile_list.append(output_fc)

    return tile_list

# Function to calculate entropy index (EI) for every polygon using Zonal Histogram
def calculate_entropy(tile_fc, raster_path, target_values, residential_values):
    try:
        start_time = time.time()
        tile_name = os.path.basename(tile_fc)
        zh_table = f"{output_gdb}\\zh_{tile_name}"
        print(f"Calculating zonal histogram on {tile_name}...")

        arcpy.sa.ZonalHistogram(
            in_zone_data=tile_fc,
            zone_field="OrigOID",
            in_value_raster=raster_path,
            out_table=zh_table,
            out_graph="",
            zones_as_rows="ZONES_AS_ROWS"
        )
        elapsed_histogram = time.time() - start_time
        print(f"Zonal histogram completed in {elapsed_histogram:.2f} seconds.")

        results = {}
        print("Processing polygons...")

        with arcpy.da.SearchCursor(zh_table, "*") as cursor:
            for row in cursor:
                label = row[1]
                orig_oid = int(label.replace("OrigO_", ""))
                pixel_values = {int(cursor.fields[col_idx].replace("CLASS_", "")) - 1: value
                                for col_idx, value in enumerate(row[2:], start=2) if value > 0}

                total_cells = sum(pixel_values.values())
                matching_cells = sum(pixel_values[val] for val in pixel_values if val in target_values)
                residential_cells = sum(pixel_values[val] for val in pixel_values if val in residential_values)

                if matching_cells == total_cells:
                    adjustment = random.randint(1, 10)
                    matching_cells -= adjustment

                Pij = matching_cells / total_cells if total_cells > 0 else 0
                Nj = max(2, len(pixel_values))
                EI = -(Pij * math.log(Pij)) / math.log(Nj) if Pij > 0 else 0
                residential_coefficient = residential_cells / total_cells if total_cells > 0 else 0
                EI_adjusted = EI * (1 + residential_coefficient)

                results[orig_oid] = EI_adjusted

        elapsed_processing = time.time() - start_time
        print(f"Finished processing polygons in {tile_name} in {elapsed_processing:.2f} seconds.")
        return results

    except arcpy.ExecuteError:
        print(f"Error in calculate_entropy for {tile_fc}:")
        print(arcpy.GetMessages(2))

    except Exception as e:
        print(f"General error in calculate_entropy for {tile_fc}: {e}")

# Function to write results to a standalone CSV file
def rejoin_results(results):
    try:
        with open(output_csv_path, mode='w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["OrigOID", "EI"])
            for orig_oid, ei_value in results.items():
                writer.writerow([orig_oid, ei_value])

        print(f"Results written to {output_csv_path}")

    except Exception as e:
        print(f"Error writing results to CSV: {e}")


## Workflow

In [12]:
def get_memory_usage():
    """Return the current memory usage in MB."""
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")

    try:
        # Step 1: Add fields
        print("Adding necessary fields to the fishnet...")
        add_fields_to_fc(fishnet_fc)

        # Step 2: Split fishnet into tiles
        print("Splitting fishnet into tiles...")
        tiles = split_fishnet_by_count(fishnet_fc, target_polys, output_gdb)
        total_tiles = len(tiles)  # Total number of tiles for progress tracking
        print(f"Created {total_tiles} tiles.")

        all_results = {}
        tile_start_time = time.time()

        # Step 3: Process each tile
        for idx, tile in enumerate(tiles, start=1):
            print(f"Processing tile {idx} of {total_tiles}: {os.path.basename(tile)}...")
            tile_results = calculate_entropy(tile, raster_path, target_values, residential_values)

            if tile_results:
                all_results.update(tile_results)

            # Tile timing and progress
            elapsed_time = time.time() - tile_start_time
            avg_time_per_tile = elapsed_time / idx
            remaining_tiles = total_tiles - idx
            estimated_time_remaining = avg_time_per_tile * remaining_tiles

            # Memory usage
            memory_usage = get_memory_usage()

            print(f"Finished tile {idx}/{total_tiles} in {elapsed_time:.2f} seconds.")
            print(f"Average time per tile: {avg_time_per_tile:.2f} seconds.")
            print(f"Estimated time remaining: {estimated_time_remaining:.2f} seconds.")
            print(f"Current memory usage: {memory_usage:.2f} MB.")
            tile_start_time = time.time()  # Reset for the next tile

        # Step 4: Write results to a CSV file
        print("Writing results to output CSV...")
        rejoin_results(all_results)

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        # Cleanup temporary files
        print("Cleaning up temporary files...")
        arcpy.env.workspace = output_gdb
        arcpy.ClearWorkspaceCache_management()

        tile_fcs = arcpy.ListFeatureClasses("Tile_*")
        for tile_fc in tile_fcs:
            arcpy.management.Delete(tile_fc)

        zonal_tables = arcpy.ListTables("zonal_table_*")
        for table in zonal_tables:
            arcpy.management.Delete(table)

        print("Cleanup complete.")
        end_time = datetime.now()
        print(f"Finished processing. Total duration: {end_time - start_time}")

Start time: 2025-01-24 16:54:09.126586
Adding necessary fields to the fishnet...
Splitting fishnet into tiles...
Created 16 tiles.
Processing tile 1 of 16: Tile_0_0...
Calculating zonal histogram on Tile_0_0...
Error in calculate_entropy for D:\ECHO\CDL\ZonalEntropyIndex\Default.gdb\Tile_0_0:
ERROR 010400: No zone is found in the ZONE input in Zonal Histogram.
Failed to execute (ZonalHistogram).

Finished tile 1/16 in 0.58 seconds.
Average time per tile: 0.58 seconds.
Estimated time remaining: 8.71 seconds.
Current memory usage: 3549.08 MB.
Processing tile 2 of 16: Tile_0_1...
Calculating zonal histogram on Tile_0_1...
Zonal histogram completed in 38.29 seconds.
Processing polygons...
Finished processing polygons in Tile_0_1 in 42.98 seconds.
Finished tile 2/16 in 42.98 seconds.
Average time per tile: 21.49 seconds.
Estimated time remaining: 300.87 seconds.
Current memory usage: 4086.34 MB.
Processing tile 3 of 16: Tile_0_2...
Calculating zonal histogram on Tile_0_2...
Zonal histogram 

In [13]:
# Cleanup temporary files
print("Cleaning up temporary files...")
arcpy.env.workspace = output_gdb

# Delete the temporary feature layer if it exists
if arcpy.Exists("fishnet_layer"):
    arcpy.management.Delete("fishnet_layer")

# Delete all temporary Tile_* feature classes
tile_fcs = arcpy.ListFeatureClasses("Tile_*")
for tile_fc in tile_fcs:
    arcpy.management.Delete(tile_fc)

# Delete all temporary zonal_table_* tables
zonal_tables = arcpy.ListTables("zonal_table_*")
for table in zonal_tables:
    arcpy.management.Delete(table)
    
# Delete all temporary zh_* tables
zh_tables = arcpy.ListTables("zh_*")
for table in zh_tables:
    arcpy.management.Delete(table)

# Clear workspace locks
arcpy.ClearWorkspaceCache_management()

print("Cleanup complete.")

Cleaning up temporary files...
Cleanup complete.
